In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset=pd.read_csv("../input/anime-recommendations-database/anime.csv")

In [ ]:
dataset

Check if some null values are here or not.

In [ ]:
dataset.isnull().sum()

As we see a very less null values inside the dataset let say:
> in Genre only 0.5% are null values
> In type only 0.2%
> In ratings 1.8%
> So remove the null values

In [ ]:
dataset=dataset.dropna()


In [ ]:
dataset

In [ ]:
dataset.shape

In [ ]:
dataset.info()

Lets Check the 2nd dataset also

In [ ]:
rating_df = pd.read_csv('../input/anime-recommendations-database/rating.csv')
rating_df

What each column represents:
> user_id: non-identifiable randomly generated user-id
> > anime_id: the anime that the user has rated
> > > rating: rating out of 10 this user has assigned (-1 if the user watched it but didn’t assign a rating)

In [ ]:
rating_df.shape

As both Anime and rating have same column that is anime_id, **So join them**

In [ ]:
df_merge=pd.merge(dataset, rating_df, on = 'anime_id')

In [ ]:
df_merge.shape

As i am going to use SVD so it cannot handle this much data. So lets reduce it 

In [ ]:
df_merge.rating_x.min()

I have decided to take out the columns with no ratings, that are represented as -1

In [ ]:
df_merge = df_merge[df_merge.rating_y != -1]

In [ ]:
df_merge.shape

As you see it almost remove the 1 lakh rows from dataset

Still too large so i convert it into small size

In [ ]:
sample = df_merge.sample(frac=.25)
sample.shape

In [ ]:
sample.dtypes

Lets Convert

In [ ]:
sample['rating_x'] = sample['rating_x'].astype(int)

In [ ]:
sample.dtypes

it seems that the rating dataframe is based on each user and their individual rating for each anime id, whereas the anime dataframe is an average overall rating from all of its viewers.
**we are going to look at how many times each rating appears in a column**

In [ ]:
ratings_x = sample['rating_x'].value_counts() #continuous
ratings_y = sample['rating_y'].value_counts() #discrete
print(ratings_x)
print(ratings_y)

Important thing for SVD is to make your variables discreet, otherwise, it’ll take alot more of your time.

In [ ]:
sample.rating_x = sample.rating_x.apply(round) 

In [ ]:
sample.head()

In [ ]:
sample.shape

****EDA****

In [ ]:
import matplotlib.pyplot as plt
ratings_sorted = sorted(list(zip(ratings_y.index, ratings_y)))
plt.bar([r[0] for r in ratings_sorted], [r[1] for r in ratings_sorted], color='blue')
plt.xlabel("Rating")
plt.ylabel("# of Ratings")
plt.title("Distribution of Ratings")
plt.show()

In [ ]:
import seaborn as sns
fig = plt.figure(figsize=(12
                          ,5))
sns.countplot(sample['type'], palette='gist_rainbow')
plt.title("Most Viewed Anime on Type", fontsize=20)
plt.xlabel("Types", fontsize=20)
plt.ylabel("Number of Views with Reviews", fontsize = 20)
plt.grid()
plt.show()

**Analysis Part**

First thing’s first! Do a base model. I started with transforming my data.

In [ ]:
from surprise import Reader
from surprise import Dataset
data = sample[['user_id', 'anime_id', 'rating_x']] #may need to do rating_x rounded and then use rating_y
reader = Reader(line_format='user item rating', sep='')

In [ ]:
anime = Dataset.load_from_df(data, reader)

In [ ]:
#train_test_split
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.prediction_algorithms import knns
from surprise.prediction_algorithms import SVD
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
trainset, testset = train_test_split(anime, test_size=.2)

In [ ]:
#INSTANTIATE the SVD and fit only the train set
svd = SVD()
svd.fit(trainset)

In [ ]:
predictions = svd.test(testset) 
accuracy.rmse(predictions)

This is what i have made .
> I know this is not the great accuracy can anyone suggest some other methods so i can try those.
>> thanks in advance

In [ ]:
#